# Catalog inspection - By Field

Perform more detailed verification on the datasets, using LSDB to inspect leaf parquet files, using spatial fields.

In [ ]:
!pip install lsdb --quiet

In [95]:
%env DRP_VERSION=w_2025_07

env: DRP_VERSION=w_2025_07


In [96]:
import os
import hats
import lsdb
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import itertools

from pathlib import Path

In [97]:
DRP_VERSION = os.environ["DRP_VERSION"]
print(f"DRP_VERSION: {DRP_VERSION}")
base_output_dir = Path(f"/sdf/data/rubin/shared/lsdb_commissioning")
raw_dir = base_output_dir / "raw" / DRP_VERSION
hats_dir = base_output_dir / "hats" / DRP_VERSION

DRP_VERSION: w_2025_07


In [98]:
fields_dir = raw_dir / "field_sizes"
fields_dir.mkdir(parents=True, exist_ok=True)

In [99]:
catalogs = [
    "diaObject",
    "diaSource",
    "diaForcedSource",
    "object",
    "source",
    "forcedSource",
]   

# Define the six fields with their coordinates
fields = {
    "ECDFS": (53.13, -28.10),  # Extended Chandra Deep Field South
    "EDFS": (59.10, -48.73),  # Euclid Deep Field South
    "Rubin_SV_38_7": (37.86, 6.98),  # Low Ecliptic Latitude Field
    "Rubin_SV_95_-25": (95.00, -25.00),  # Low Galactic Latitude Field
    "47_Tuc": (6.02, -72.08),  # 47 Tuc Globular Cluster
    "Fornax_dSph": (40.00, -34.45)  # Fornax Dwarf Spheroidal Galaxy
}

# Define the radius for selecting sources
selection_radius_arcsec = 2.0 * 3600  # 2-degree radius

# Define bands
bands = ["u", "g", "r", "i", "z", "y"]

In [100]:
cat = lsdb.read_hats(hats_dir / "source")
cat

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,decl,raErr,decErr,ra_dec_Cov,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,sizeExtendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,apFlux_35_0_flag,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,apFlux_50_0_flag,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,normCompTophatFlux_flag,normCompTophatFlux_instFlux,normCompTophatFlux_instFluxErr,extendedness_flag,sizeExtendedness_flag,footprintArea_value,invalidPsfFlag,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,sourceId,day_obs,psfMag,psfMagErr,midpointMjdTai,Norder,Dir,Npix
npartitions=117,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 5, Pixel: 32",double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[py

In [101]:
## What are the columns of interest for the results? Everything numeric!
print("starting column count", len(cat._ddf.meta.columns))
column_names = list(cat._ddf.meta.select_dtypes(include=np.number))
# Exclude HATS-added columns
column_names = [c for c in column_names if c not in ["_healpix_29", "Norder", "Dir", "Npix"]]
column_names = [c for c in column_names if not c.endswith("Id")]
column_names = [c for c in column_names if "Mag" not in c]
print("effective column count", len(column_names))

starting column count 172
effective column count 79


In [117]:
# Function to compute statistics
def get_stats(df, stat_columns, out_columns):
    stats = {col: 0 for col in out_columns}

    # Exclude rows where sky_source is True
    df = df[df["sky_source"] == False]  

    for band in bands:
        mask = df["band"] == band  # Filter by band
        if mask.sum() > 0:  # Ensure there are sources in this band
            for col in stat_columns:
                stats[f"mean_{col}_{band}"] = np.nanmean(df.loc[mask, col])
                stats[f"min_{col}_{band}"] = np.nanmin(df.loc[mask, col])
                stats[f"max_{col}_{band}"] = np.nanmax(df.loc[mask, col])
            stats[f"len_{band}"] = len(df.loc[mask, "x"]) - np.count_nonzero(np.isnan(df.loc[mask, "x"]))
    
    return pd.DataFrame([stats])  # Convert to DataFrame

# Dictionary to store results
all_results = {}

meta = {
    **{f"mean_{column}_{band}": "f8" for (column, band) in itertools.product(column_names, bands)},
    **{f"min_{column}_{band}": "f8" for (column, band) in itertools.product(column_names, bands)},
    **{f"max_{column}_{band}": "f8" for (column, band) in itertools.product(column_names, bands)},
    **{f"len_{band}": "i8" for band in bands}
}

In [ ]:
# Loop through each field and perform cone search + computation
for field_name, (ra, dec) in tqdm(fields.items()):
    # Perform cone search for the given field
    field_cat = cat.cone_search(ra=ra, dec=dec, radius_arcsec=selection_radius_arcsec)
    
    # Compute statistics
    result = field_cat.map_partitions(
        get_stats,
        meta=meta,
        stat_columns=column_names,
        out_columns=meta.keys(),
    ).compute()

    # Compute weighted statistics for each band separately
    weighted_stats = {}
    for (column, band) in itertools.product(column_names, bands):
        len_band = np.nansum(result[f"len_{band}"])
        if len_band:
            mean_col_name = f"mean_{column}_{band}"
            min_col_name = f"min_{column}_{band}"
            max_col_name = f"max_{column}_{band}"
            
            # Weighted mean
            weighted_stats[mean_col_name] = np.nansum(result[mean_col_name] * result[f"len_{band}"]) / len_band
            
            # Weighted min (minimum value among sources)
            weighted_stats[min_col_name] = np.nanmin(result[min_col_name])  # Min is not weighted
            
            # Weighted max (maximum value among sources)
            weighted_stats[max_col_name] = np.nanmax(result[max_col_name])  # Max is not weighted

    # Store the weighted statistics for this field
    all_results[field_name] = weighted_stats

# Convert to DataFrame for better visualization
weighted_stats_df = pd.DataFrame.from_dict(all_results, orient="index")

  0%|                                                                              | 0/6 [00:00<?, ?it/s]

In [116]:
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.4f}'.format 
weighted_stats_df.T

,ECDFS,EDFS,Rubin_SV_95_-25,Rubin_SV_38_7,47_Tuc,Fornax_dSph
mean_coord_ra_u,53.1407,59.1284,94.9388,NaN,NaN,NaN
min_coord_ra_u,0.0000,58.2195,0.0000,NaN,NaN,NaN
max_coord_ra_u,53.8704,60.0737,95.7620,NaN,NaN,NaN
mean_coord_ra_g,53.1341,59.1475,95.0031,37.8649,5.8871,39.9449
min_coord_ra_g,52.3521,58.1336,94.2562,36.9180,0.0000,0.0000
max_coord_ra_g,53.9326,60.1480,95.7268,38.8860,7.5204,40.5948
mean_coord_ra_r,53.1424,59.1195,95.0453,37.8669,6.4768,40.0237
min_coord_ra_r,52.3802,58.0926,0.0000,36.9173,3.9606,39.3045
max_coord_ra_r,53.9023,60.0856,95.7854,38.8879,8.1547,40.7112
mean_coord_ra_i,53.1109,59.1085,95.0340,37.8873,5.9603,39.9912
